# Notebook 2 – Trader Leverage & Sentiment Deep-Dive
Use this notebook after running Notebook 1. It inspects the CSV outputs (rankings, clusters, regime metrics, leverage alerts) generated inside `ds_Advay_Sinha/csv_files`.


**What you can do here**
- Validate leverage and ROI behavior for each sentiment bucket.
- Merge rankings with cluster labels to craft qualitative notes.
- Produce extra charts or tables for the PDF/README if reviewers need visuals.


In [ ]:
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
    print('Drive mounted. Update DATA_ROOT below if needed.')
except ModuleNotFoundError:
    print('Running outside Colab. Proceed to the next cell.')


In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')


In [ ]:
DATA_ROOT = Path('/content/drive/MyDrive/ds_Advay_Sinha')
if not DATA_ROOT.exists():
    DATA_ROOT = Path.cwd()
CSV_DIR = DATA_ROOT / 'csv_files'
OUTPUT_DIR = DATA_ROOT / 'outputs'
print('Using data root:', DATA_ROOT)


In [ ]:
rankings = pd.read_csv(CSV_DIR / 'trader_rankings.csv')
clusters = pd.read_csv(CSV_DIR / 'trader_behavior_clusters.csv')
regime_metrics = pd.read_csv(CSV_DIR / 'trader_regime_metrics.csv')
leverage_alerts = pd.read_csv(CSV_DIR / 'leverage_alerts.csv')
model_metrics = pd.read_json(CSV_DIR / 'model_metrics.json')
strategy_metrics = pd.read_json(CSV_DIR / 'strategy_metrics.json')
print('Rankings:', rankings.shape, '| Clusters:', clusters.shape)


In [ ]:
rankings_clusters = rankings.merge(clusters, on='account', how='left')
rankings_clusters.head()


In [ ]:

plt.figure(figsize=(10, 5))
top_accounts = rankings_clusters.head(20)
sns.barplot(data=top_accounts, x='account', y='composite_score', hue='kmeans_cluster')
plt.xticks(rotation=90)
plt.title('Top Accounts by Composite Score and KMeans Cluster')
plt.legend(title='KMeans Cluster')
plt.tight_layout()
plt.show()


In [ ]:

dbscan_counts = (
    clusters['dbscan_cluster']
    .value_counts(dropna=False)
    .reset_index()
    .rename(columns={'index': 'dbscan_cluster', 'dbscan_cluster': 'account_count'})
)
dbscan_counts


In [ ]:
plt.figure(figsize=(8, 4))
sns.barplot(data=regime_metrics, x='sentiment_classification', y='roi_mean')
plt.title('Average ROI by Sentiment Classification')
plt.ylabel('ROI mean')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 4))
sns.barplot(data=regime_metrics, x='sentiment_classification', y='abs_leverage_mean')
plt.title('Average Leverage Ratio by Sentiment Classification')
plt.ylabel('Leverage mean')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 4))
if not leverage_alerts.empty:
    sns.barplot(data=leverage_alerts.sort_values('max_leverage_ratio', ascending=False), x='account', y='max_leverage_ratio')
    plt.xticks(rotation=90)
    plt.title('Accounts Requiring Leverage Review')
    plt.tight_layout()
else:
    print('No leverage alerts were triggered.')


In [ ]:
model_metrics.T


In [ ]:
strategy_metrics.T


### Tips
- Export additional figures via `plt.savefig(OUTPUT_DIR / 'your_chart.png')` for inclusion in the submission.
- Add commentary cells describing notable traders or leverage alerts so the PDF can reference them directly.
